### Thymus human spatial atlas dissociated dataset loading
### notebook to read all single cell data before integration 
### preprocessing - for all datasets, supp was removed with cellbender - see "preprocessing" for details
### last edit 02/07/24


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
import scanpy as sc
import scrublet as sl
import matplotlib as mpl
import numpy as np
import pandas as pd
import os
import anndata as ad
import scvi
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white', color_map='viridis')
sc.logging.print_header()

In [ ]:
Path = ''
os.chdir(Path)
meta = pd.read_csv('')
meta.columns = meta.columns.str.strip()
meta

# Read all other data and merge with demultiplexd nuclei 

In [ ]:
# read the rest of the libraries and run scrublet 
import warnings 
warnings.simplefilter("ignore")

In [ ]:
X = pd.DataFrame(np.empty((0,0)))
obs = pd.DataFrame(index=X.index)
var = pd.DataFrame(index=X.columns) 
X = X.values

AnnData = ad.AnnData(X=X, obs=obs, var=var)
data = {}
for idx,f in enumerate(meta.iloc[:,1]):
#     if f!='sample6':
        print(str(idx))
        print(f)
        results_file = meta['path_cellbender_gex'][idx]+'/'+meta['library'][idx]+'_filtered.h5'
        print(results_file)
        adata = sc.read_10x_h5(results_file)
        adata.var_names_make_unique()
        sc.pp.filter_cells(adata, min_genes=400)
        print(meta['library'][idx])
        # add metadata 
        adata.obs['barcode'] = adata.obs.index
        adata.obs['sample_barcode'] = meta['sample'][idx] + '-' + adata.obs.index
        adata.obs.index = meta['library'][idx] + '-' + adata.obs.index
        adata.obs['batch'] = idx+meta.shape[0] # fix batch 
        adata.obs['chemistry_simple'] = meta['chemistry_simple'][idx]
        adata.obs['age_group'] = meta['age_group'][idx]
        adata.obs['age_numeric'] = meta['age_cont'][idx]
        adata.obs['sampleID_TCRab'] = meta['sample_id_tcrab'][idx]
        adata.obs['path_TCRab'] = meta['path_tcrab'][idx]
        adata.obs['path_cellbender_gex'] = meta['path_cellbender_gex'][idx]
        adata.obs['study'] = meta['study'][idx]
        adata.obs['study_group'] = meta['study_group'][idx]
        adata.obs['sample'] = meta['sample'][idx]
        adata.obs['enrichment'] = meta['sort'][idx]
        adata.obs['donor'] = meta['donor'][idx]
        adata.obs['cite'] = meta['cite'][idx]
        print(adata.shape)
        # doublet detection     
        sc.external.pp.scrublet(adata)
        AnnData = ad.concat([AnnData,adata],join='outer')

        print(AnnData.shape)
AnnData.obs.index.names = ['barcode']

# QC MT and counts

In [ ]:
# qc
# mitochondrial genes
AnnData.var['mt'] = AnnData.var_names.str.startswith('MT-') 
# ribosomal genes
AnnData.var['ribo'] = AnnData.var_names.str.startswith(("RPS","RPL"))
# hemoglobin genes.
AnnData.var['hb'] = AnnData.var_names.str.contains(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(AnnData, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)
AnnData.obs

In [ ]:
mito_genes = AnnData.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
AnnData.obs['percent_mt2'] = np.sum(AnnData[:, mito_genes].X, axis=1).A1 / np.sum(AnnData.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
AnnData.obs['n_counts'] = AnnData.X.sum(axis=1).A1

In [ ]:
sc.pl.violin(AnnData, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb','doublet_score'],
             jitter=0.1, multi_panel=True)

In [ ]:
AnnData.obs['barcode_sequence'] = AnnData.obs['barcode']

In [ ]:
AnnData.obs = AnnData.obs.drop(columns='barcode')

In [ ]:
AnnData

In [ ]:
# remove poor quality cells
AnnData = AnnData[AnnData.obs.n_genes_by_counts < 6500, :]
AnnData = AnnData[AnnData.obs.pct_counts_mt < 6, :]
AnnData = AnnData[AnnData.obs.pct_counts_ribo > 5, :]
AnnData.obs['cite'].value_counts()

In [ ]:
AnnData.var_names_make_unique()
AnnData.obs

# SCVI integration 

In [ ]:
AnnData_r = AnnData.copy()  # generate a temp AnnData 
AnnData_r.layers["counts"] = AnnData_r.X.copy() # preserve counts
AnnData_r.obs = AnnData.obs # recover counts
AnnData_r.var = AnnData.var # recover var names 
AnnData_r

## step 1 - remove cc and mt genes 

In [ ]:
genes = {'cc_genes' : ['C1orf112', 'KDM1A', 'ARHGAP33', 'CDC27', 'AC004381.6', 'DBF4', 'PRSS21', 'E2F2', 'BAZ1B', 'NCAPD2', 'ANLN', 'BRCA1', 'UBR7', 'DDX11', 'TACC3', 'POLA2', 'DEPDC1', 'NCAPH2', 'HMGB3', 'CENPQ', 'DEPDC1B', 'SPDL1', 'TDP1', 'DSG2', 'NOP16', 'RFC2', 'RAD51', 'POLQ', 'THOC3', 'MRTO4', 'C4orf27', 'POLD1', 'TM7SF3', 'MCM10', 'MTHFD2', 'YBX1', 'ASPM', 'PRR11', 'NUCKS1', 'EXOC5', 'TCOF1', 'RAD18', 'TRIP13', 'DAZAP1', 'HLTF', 'PDCD2', 'SMC1A', 'HMMR', 'MCM2', 'NUP37', 'GTSE1', 'WDR62', 'MCM6', 'MLH1', 'UNG', 'SPAG5', 'UBE2T', 'EXOSC5', 'FAM76B', 'POLD3', 'KIF22', 'PUM3', 'RBL1', 'NDC80', 'HSPB11', 'OXCT1', 'SEH1L', 'ORC1', 'RAD54L', 'NOP14', 'GMCL1', 'AURKA', 'TPX2', 'BIRC5', 'KIF4A', 'FH', 'ORC6', 'G2E3', 'WDR76', 'PHGDH', 'CLSPN', 'NUP50', 'CDC45', 'CDC6', 'CBX5', 'MSH2', 'FKBP5', 'ACOT7', 'CDC7', 'RANBP1', 'CENPM', 'HMGXB4', 'MCM5', 'RANGAP1', 'POLE2', 'CDKN3', 'ERH', 'MTHFD1', 'VRK1', 'EMC9', 'GINS1', 'MYBL2', 'CDC25B', 'PLCB4', 'FAM83D', 'CEP76', 'RBBP8', 'POLA1', 'NUP93', 'CTCF', 'CCP110', 'CEP152', 'OIP5', 'MYEF2', 'MCM4', 'KCTD9', 'RNASEH2A', 'SGTA', 'ASF1B', 'LIG1', 'H2AFV', 'GARS', 'NUDT1', 'RPA3', 'EZH2', 'RHEB', 'SMC3', 'UBE2S', 'RAD51C', 'GALK1', 'C1QBP', 'BLMH', 'TMEM97', 'GAR1', 'NEIL3', 'WHSC1', 'NCAPG', 'CCDC34', 'CCDC86', 'MAGOHB', 'FOXM1', 'GPN3', 'RAD51AP1', 'RFC5', 'NUP107', 'TIMELESS', 'GAPDH', 'CDCA3', 'ENO2', 'FBXO5', 'MCM3', 'BAG2', 'GMNN', 'TTK', 'HMGCS1', 'BRD8', 'KIF20A', 'LMNB1', 'NUP155', 'H2AFY', 'SMC4', 'ECT2', 'CENPA', 'FHL2', 'MSH6', 'DHCR24', 'LRRC42', 'WDR77', 'SRM', 'MAD2L2', 'UCHL5', 'TTF2', 'SLC2A1', 'EBNA1BP2', 'CDC20', 'STMN1', 'NEK2', 'CENPF', 'KIF14', 'CASP8AP2', 'CTNNAL1', 'WDR34', 'PHF19', 'ACYP1', 'EPCAM', 'HELLS', 'CENPL', 'ACAT2', 'MASTL', 'EXOSC8', 'TMPO', 'NFYB', 'NCAPH', 'MND1', 'KIF18A', 'GPSM2', 'GTF3A', 'MRPS2', 'CCDC18', 'CISD1', 'ZWINT', 'CIT', 'DDX39A', 'CENPK', 'CDK2', 'TUBA1B', 'STIL', 'HJURP', 'EXOSC9', 'CKS2', 'ACSL3', 'CSE1L', 'HIST1H1D', 'HIST1H1A', 'DEK', 'GOT2', 'MGME1', 'MCM8', 'HNRNPR', 'BCL2L12', 'TRAP1', 'DLGAP5', 'YEATS4', 'PKMYT1', 'SRD5A3', 'PAICS', 'HAT1', 'KNSTRN', 'DUT', 'E2F8', 'FAM64A', 'MIS18BP1', 'SGOL1', 'RPS4Y1', 'LDLR', 'TOMM40', 'ATP8B3', 'LSM4', 'DNMT1', 'GINS2', 'TUBG1', 'PSMC3IP', 'THOC6', 'TOP2A', 'ENOSF1', 'RAN', 'RPA1', 'PCNA', 'NASP', 'RFC3', 'SLF1', 'LRRCC1', 'CCNB1', 'PSRC1', 'TMEM106C', 'LDHA', 'DDB2', 'CDCA8', 'TPGS2', 'SLC43A3', 'C9orf40', 'TROAP', 'ESPL1', 'NTPCR', 'CKAP2', 'BORA', 'NUDT15', 'BRIP1', 'ACTL6A', 'TXN', 'ODF2', 'SMC2', 'CDK5RAP2', 'ANP32B', 'DSCC1', 'ARHGEF39', 'PPIL1', 'TCF19', 'NRM', 'GGH', 'BUD13', 'CTDSPL2', 'NUSAP1', 'KIF23', 'CASC5', 'CENPO', 'KIF11', 'CEP55', 'KIF20B', 'DNA2', 'BARD1', 'MNS1', 'ZGRF1', 'CENPE', 'HADH', 'SLC39A8', 'NEDD1', 'BRCA2', 'DIAPH3', 'PIF1', 'FANCI', 'TICRR', 'SAMD1', 'SAE1', 'PLK4', 'ITGB3BP', 'KIF2C', 'UCK2', 'NUF2', 'ANP32E', 'DTL', 'ILF2', 'C1orf35', 'HNRNPLL', 'CHAC2', 'POLR2D', 'CDCA7', 'FANCD2', 'CCNA2', 'SKP2', 'PRIM2', 'MMS22L', 'PM20D2', 'MTFR2', 'CDCA5', 'NCAPG2', 'PRPS1', 'GINS4', 'HAUS6', 'CEP78', 'GTF3C5', 'MKI67', 'IMMP1L', 'SSRP1', 'INCENP', 'CCDC15', 'CHEK1', 'DSN1', 'HIRIP3', 'HMGA2', 'TEX30', 'NCAPD3', 'CENPU', 'CENPJ', 'SPC25', 'SUV39H2', 'CENPH', 'CARHSP1', 'CLGN', 'CCSAP', 'BUB3', 'SLFN13', 'SKA1', 'SLC16A1', 'TMEM237', 'ADK', 'FAM122B', 'AIFM1', 'ATAD2', 'SASS6', 'BUB1B', 'CCNB2', 'PSMG3', 'FAIM', 'CDC25C', 'MIS18A', 'ATP5G1', 'IGF2BP1', 'CHAF1B', 'NAE1', 'CCDC58', 'C21orf58', 'ZNF714', 'GPATCH4', 'YDJC', 'RACGAP1', 'SPC24', 'POLR3K', 'CCNF', 'ASRGL1', 'USP1', 'KIAA1524', 'SGOL2', 'TOPBP1', 'KIF15', 'WDR43', 'RFC4', 'RPL39L', 'RNF168', 'H2AFZ', 'CDC25A', 'POC1A', 'HMGB2', 'MAD2L1', 'PTTG1', 'RAD21', 'PSIP1', 'GKAP1', 'MELK', 'SLC7A3', 'SKA3', 'LRR1', 'HPRT1', 'E2F7', 'JAM3', 'BRD7', 'CENPN', 'LEO1', 'WEE1', 'MCM7', 'CENPV', 'SAAL1', 'FAM111A', 'KIAA0101', 'PLK1', 'XRCC6BP1', 'SNRPD1', 'RRM1', 'CDT1', 'CHAF1A', 'C19orf48', 'NUDT8', 'PRDX2', 'TK1', 'SAC3D1', 'PBK', 'HIST1H1E', 'DTYMK', 'RFWD3', 'FEN1', 'ING2', 'CDKN2AIP', 'SNRNP48', 'USP39', 'CKAP2L', 'BUB1', 'ELP5', 'CDK1', 'CDCA4', 'SHCBP1', 'PGAM1', 'ESCO2', 'RHNO1', 'CTPS1', 'RRM2', 'THOP1', 'C5orf34', 'POP7', 'ZNF738', 'CKS1B', 'GMPPB', 'CNP', 'CBX2', 'C16orf91', 'EXO1', 'ZWILCH', 'ANKRD36C', 'UBE2C', 'CKAP5', 'CCNE2', 'CCDC14', 'RMI2', 'TOMM5', 'RUVBL1', 'ATAD5', 'LMNB2', 'TYMS', 'PXMP2', 'SHMT1', 'POLE', 'PAWR', 'PIDD1', 'GSG2', 'GEN1', 'RMI1', 'AURKB', 'RCC1', 'PPA1', 'HIST3H2A', 'MRPS23', 'RTKN2', 'IDH2', 'KPNA2', 'CEP97', 'GLRX5', 'SKA2', 'CEP57L1', 'PSMG1', 'SFXN4', 'ALYREF', 'TRAIP', 'LIN9', 'KMT5A', 'PGP', 'CMSS1', 'HIST2H2AC', 'HIST1H1B', 'KNTC1', 'CDCA2', 'H1FX', 'SIVA1', 'PARPBP', 'KIF18B', 'ERCC6L', 'PTMA', 'HSPA14', 'FANCA', 'ARHGAP11B', 'TUBB4B', 'CENPP', 'H2AFX', 'FAM72B', 'FAM221A', 'FAM111B', 'HN1', 'HMGB1', 'TUBB', 'IARS', 'ERI2', 'HIST1H4C', 'BLM', 'PRIM1', 'HMGN5', 'SVIP', 'TFDP1', 'TXNRD1', 'WDHD1', 'EIF1AY', 'ARHGAP11A', 'HMGN2', 'DCAF12', 'PRC1', 'CCDC167', 'CENPW', 'GPANK1', 'MZT1', 'C4orf46', 'KLHL23', 'MXD3', 'DNAJC9', 'IFRD2', 'CKLF', 'FANCG', 'DHFR', 'DLEU2', 'AP000251.3', 'KIFC1', 'CRNDE', 'C8orf88', 'PRKDC', 'BOP1', 'RP11-196G18.23', 'CTC-260E6.6', 'NCBP2-AS2', 'HIST1H3G', 'HIST1H2AH', 'UHRF1', 'RDM1', 'HIST1H2AM'],
'T_exhaust' : 'GATA3,IKZF2,SUDS3,PDCD1,LAG3,HAVCR2,TNFRSF9,TOX,CD81,CREB3L2,SLAMF6,TNFSF14,CD28,CD160,BTLA,CD200,CD27'.split(','),
'T_naive' : 'SELL,PECAM1,CCR7,CD27,CD28'.split(','), #45RA 
'T_centralM' : 'CCR7,CD27,CD28'.split(','), #45RO
'T_effectorM' : 'CD27,CD28,PDCD1'.split(','), #45RO
'T_EMRA' : 'PDCD1,B3GAT1,KLRG1'.split(','), #45RA
'T_residentM' : 'CXCR6,CXCR3,CCR5,CCR9,CCR4,CCR8,CCR10'.split(','), #factors specific for guidance to each organ
'MastOrILC2' : 'GATA2,IL4'.split(','),
'eosinophil' : 'CCR3,SIGLEC8,ITGAM'.split(','),
'mait' : 'SLC4A10,TRAV1-2'.split(','),
'liverHepaticStellateCell' : 'LRAT'.split(','),
'kidneyReninCells' : 'REN,NPY1R'.split(','),
'humanB1' : 'SPN,MS4A1,CD27,IGHM,CD5'.split(','),
'humanB1VDJ' : 'IGHV3-74,IGHV1-18,IGHV1-2,IGHJ6'.split(','),
'mouseB1metabolism' : ['HK2','ACLY', 'ACAT1','ACACA','ACSL1','ACACB','CPT1B','PLIN3','SLC2A1','LDHA','SREBF2','HK1','PDLIM1'],
'autoimmune' : 'TNFSF13B,IL10,IFIH1'.split(','),
'microglia' : 'TREM2,TMEM119,GPR34'.split(','),
'NK_CD56dim_mature' : 'FCGR3A,PRF1'.split(','),
'NK_CD56bright_naive' : 'NCAM1,SELL,XCL1'.split(','),
'B_naive' : 'CD37,IGHD'.split(','),
'B_activated' : 'CD27,IGHM'.split(','),
'Granulocyte' : 'S100A8,S100A9'.split(','),
'macrophage_M1': 'IL1B,TNF,IL6,NOS2,CD86,CD40,FCGR3A'.split(','),
'macrophage_M2': 'MRC1,ARG1,TREM2,CD163,IL10,IL4'.split(',')}


In [ ]:
cc_genes = genes['cc_genes'] # get cc genes from the list above 
# cc_genes

In [ ]:
def remove_selected_genes(adata,lst,remove_also_mt=True):
    for g in lst:
        adata = adata[:,~(adata.var_names==g)]
    if remove_also_mt:
        adata = adata[:,~(adata.var_names.str.startswith('MT-'))]
    return adata

AnnData_r = remove_selected_genes(AnnData_r,cc_genes,remove_also_mt=True)  

In [ ]:
# sanity check that there are not mt genes 
mtGenes = np.where(AnnData_r.var_names.str.startswith('MT-'))
print(mtGenes)

In [ ]:
AnnData_r

In [ ]:
del AnnData

In [ ]:
AnnData_r.obs = AnnData_r.obs.astype({'batch': 'category'}) # fix categorial batch 
sc.pp.normalize_total(AnnData_r)
sc.pp.log1p(AnnData_r)

In [ ]:
sc.pp.highly_variable_genes(
    AnnData_r,
    n_top_genes=5000,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
    batch_key="batch",
    span=1,
)

In [ ]:
sc.set_figure_params(figsize=[3,3])
sc.pl.highly_variable_genes(AnnData_r)

# Move to GPU instance for SCVI

In [ ]:
warnings.resetwarnings
scvi.model.SCVI.setup_anndata(AnnData_r, layer="counts", batch_key="batch", \
                        categorical_covariate_keys=["age_group",'study','chemistry_simple','enrichment'])
vae = scvi.model.SCVI(AnnData_r,n_layers=2, n_latent=30,)
vae.train(max_epochs=300,batch_size=2000)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(vae.history['elbo_train'][:]);
plt.xlim(0, len(vae.history['elbo_train']));

In [ ]:
AnnData.obsm["X_scVI"] = vae.get_latent_representation() # return batch corrections to original anndata
sc.pp.neighbors(AnnData, use_rep="X_scVI")

In [ ]:
AnnData

In [ ]:
sc.tl.umap(AnnData,min_dist=0.1)

In [ ]:
sc.set_figure_params(dpi=200,fontsize=10,)
sc.pl.umap(AnnData,color=['donor','pct_counts_mt','doublet_score','pct_counts_ribo'],size=0.5,cmap='jet',ncols=2,wspace=1)

In [ ]:
sc.set_figure_params(dpi=300,fontsize=10,)
sc.pl.umap(AnnData,color=['study','age_group','chemistry_simple','n_genes_by_counts'],size=0.1,cmap='jet',ncols=2,wspace=1)

In [ ]:
sc.pl.umap(AnnData,color=['study'],groups='HTSA_Ghent',size=0.1,cmap='jet',ncols=2,wspace=1)

In [ ]:
sc.tl.louvain(AnnData, resolution=0.5, key_added="louvian_05")
sc.tl.leiden(AnnData, resolution=0.5, key_added="leiden_05")
sc.tl.leiden(AnnData, resolution=1, key_added="leiden_1")
sc.tl.leiden(AnnData, resolution=2, key_added="leiden_2")

In [ ]:
sc.set_figure_params(dpi=200,fontsize=10,)
sc.pl.umap(AnnData,color=['louvian_05','leiden_05'],size=0.5,cmap='jet',ncols=2,wspace=1)

# Get annotations form previuos object 

In [ ]:
anno_to_map = ['annotation_level_4', 'park_2020_cell_type', 'panfetal_2022_cell_type', 'cell_type_level_4',
               'cell_type_level_3', 'cell_type_level_2', 'cell_type_level_1', 'cell_type_level_0',
                'pseudotime_nhood_vdj', 'prob_CD8+T_nhood_vdj', 'prob_CD4+T_nhood_vdj','anno_CITE_4v5']

In [ ]:
# migrate annotations form old object 
for col in anno_to_map:
    AnnData.obs[col] = AnnOld.obs[col]
    AnnData.obs[col] = AnnData.obs[col].astype('category')
    

In [ ]:
AnnData.obs['anno_CITE_4v5'].value_counts()

In [ ]:
sc.set_figure_params(dpi=200,figsize=[10,10],fontsize=6)
sc.pl.umap(AnnData,color=['anno_CITE_4v5'],size=2,cmap='jet',ncols=2,wspace=0.5,legend_loc='on data')